In [56]:
import os
import pandas as pd
import numpy as np
from pandas.tseries.offsets import BDay

PERIOD = 3
LOT = 25000
GAIN_CAP = 0.1
INCLUDE_RISK_FREE = True
HOLDING_PERIOD_CAP = 5  #business days
#Trip = ENTRY,EXIT
#Action = BUY,SELL
#Status = FILLED, PENDING, CANCELED, FORCED
blotter = pd.DataFrame(
    columns=['Date', 'Symbol', 'Trip', 'Action', 'Price', 'Size', 'Status'])

# Historical Market Data

In [57]:
file_path = os.getenv("ITA_DATA_PATH")
risk_free_path = os.getenv("RISK_FREE_DATA_PATH")
hist = pd.read_csv(f'{file_path}\data.csv', parse_dates=True, index_col='Date')
risk_free_hist = pd.read_csv(f"{risk_free_path}\\risk_free.csv", parse_dates=True, index_col='Date')

risk_free_hist['interest_rate'] = risk_free_hist['interest_rate'] / 100
hist = hist.iloc[::-1]
hist['log_ret_AMZN'] = np.log(hist['amzn_Close']) - np.log(hist['amzn_Close'].shift(1))
hist['log_ret_WMT'] = np.log(hist['wmt_Close']) - np.log(hist['wmt_Close'].shift(1))
hist['corr_coef'] = hist['amzn_Close'].rolling(PERIOD).corr(hist['wmt_Close'])
hist = hist.merge(risk_free_hist, how='left', on='Date')

# Get Position Direction

In [58]:
# This is another logic to decide if we should go long or short
# It's only based on the number of negative returns in a period of time (no tie-breaker)
def get_position_direction(slice_hist):
    amzn_neg_count = np.sum((slice_hist['log_ret_AMZN'].values < 0))
    wmt_neg_count = np.sum((slice_hist['log_ret_WMT'].values < 0))
    amzn_direction = 'BUY' if (amzn_neg_count >= wmt_neg_count) else 'SELL'
    wmt_direction = 'SELL' if (amzn_direction == 'BUY') else 'BUY'
    return {'AMZN': amzn_direction, 'WMT': wmt_direction}

In [59]:
# This logic decides if we should go long or short
# It's based on the volume of stocks that had negative returns (it's tie-breaker)
def get_position_direction2(slice_hist):
    amzn_total_volume = slice_hist['amzn_Volume'].sum()
    amz_negative_volume = np.where(slice_hist['log_ret_AMZN'] < 0, slice_hist['amzn_Volume'], 0).sum()
    amz_negative_pct = amz_negative_volume / amzn_total_volume

    wmt_total_volume = slice_hist['wmt_Volume'].sum()
    wmt_negative_volume = np.where(slice_hist['log_ret_WMT'] < 0, slice_hist['wmt_Volume'], 0).sum()
    wmt_negative_pct = wmt_negative_volume / wmt_total_volume

    amzn_direction = 'BUY' if (amz_negative_pct >= wmt_negative_pct) else 'SELL'
    wmt_direction = 'SELL' if (amzn_direction == 'BUY') else 'BUY'

    return {'AMZN': amzn_direction, 'WMT': wmt_direction}

# Create Entry Trades

In [60]:
def create_entry_trades(biz_date):
    from_date = biz_date - BDay(PERIOD - 1)
    to_date = biz_date
    slice_hist = hist.loc[from_date: to_date, :]

    trade_date = hist[hist.index > biz_date].index[0]  #get next date
    pos_direction = get_position_direction2(slice_hist)
    amzn_open_price = hist.loc[trade_date]['amzn_Open']
    amzn_size = LOT / amzn_open_price
    wmt_open_price = hist.loc[trade_date]['wmt_Open']
    wmt_size = LOT / wmt_open_price

    amzn_entry_trade = {
        "Date": trade_date,
        "Symbol": 'AMZN',
        "Trip": 'ENTRY',
        "Action": pos_direction['AMZN'],
        "Price": amzn_open_price,
        "Size": round(amzn_size, 4),  #rounding based on IB's minimum fractional size
        "Status": 'FILLED'
    }
    wmt_entry_trade = {
        "Date": trade_date,
        "Symbol": 'WMT',
        "Trip": 'ENTRY',
        "Action": pos_direction['WMT'],
        "Price": wmt_open_price,
        "Size": round(wmt_size, 4),
        "Status": 'FILLED'
    }
    return pd.DataFrame([amzn_entry_trade, wmt_entry_trade])

# Create Exit Trades

In [61]:
#The exit trades are LMT orders created right after entry trades with "Date" as the expiration date
def create_exit_trade(entry_trades_lcl, symbol):
    entry_data = entry_trades_lcl[entry_trades['Symbol'] == symbol].reset_index()
    entry_date = entry_data.at[0, 'Date']
    exit_date = entry_date + BDay(HOLDING_PERIOD_CAP)

    entry_action = entry_data.at[0, 'Action']
    exit_action = 'SELL' if entry_action == 'BUY' else 'BUY'

    entry_price = entry_data.at[0, 'Price']
    exit_price = None
    risk_free = hist.loc[entry_date]['interest_rate']
    if exit_action == 'SELL':
        exit_price = entry_price * (1 + GAIN_CAP + risk_free) if INCLUDE_RISK_FREE else  entry_price * (1 + GAIN_CAP)
    else:
        exit_price = entry_price * (1 - GAIN_CAP - risk_free) if INCLUDE_RISK_FREE else entry_price * (1 - GAIN_CAP)

    exit_size = entry_data.at[0, 'Size']

    return {
        "Date": exit_date,
        "Symbol": symbol,
        "Trip": 'EXIT',
        "Action": exit_action,
        "Price": round(exit_price, 2),
        "Size": exit_size,
        "Status": 'PENDING'
    }

In [62]:
def create_exit_trades(entry_trades_lcl):
    amzn_exit_trade = create_exit_trade(entry_trades_lcl, 'AMZN')
    wmt_exit_trade = create_exit_trade(entry_trades_lcl, 'WMT')
    return pd.DataFrame([amzn_exit_trade, wmt_exit_trade])

In [63]:
# The forced trades are MKT orders created when the exit trades are not filled within the holding period cap.
# It uses the Close Price of the next day following the exit trade's date
def create_forced_trade(biz_date, symbol, action, size, market_data):
    column_prefix = symbol.lower()
    close_price_column_name = f"{column_prefix}_Close"
    return {
        "Date": biz_date,
        "Symbol": symbol,
        "Trip": 'EXIT',
        "Action": action,
        "Price": market_data[close_price_column_name],
        "Size": size,
        "Status": 'FORCED'
    }

In [64]:
def should_force_close_position(position, biz_date):
    if position['Date'] < biz_date:
        return True

In [65]:
def should_close_position(position, biz_date):
    column_prefix = position['Symbol'].lower()
    high_price_column_name = f"{column_prefix}_High"
    low_price_column_name = f"{column_prefix}_Low"
    if position['Action'] == 'SELL' and hist.loc[biz_date][high_price_column_name] >= position['Price']:
        return True
    if position['Action'] == 'BUY' and hist.loc[biz_date][low_price_column_name] <= position['Price']:
        return True
    return False

# Run Back Test

In [66]:
for index, today_market_data in hist.iterrows():
    business_date = index
    pending_exit_trades = blotter[(blotter['Trip'] == 'EXIT') & (blotter['Status'] == 'PENDING')]
    current_position_status = 'CLOSED' if  pending_exit_trades.empty else 'OPEN'

    if current_position_status == 'CLOSED' and today_market_data['corr_coef'] < 0:
        entry_trades = create_entry_trades(business_date)
        blotter = pd.concat([blotter, entry_trades], ignore_index=True)

        exit_trades = create_exit_trades(entry_trades)
        blotter = pd.concat([blotter, exit_trades], ignore_index=True)
    elif current_position_status == 'OPEN':
        for i, pending_trade in pending_exit_trades.iterrows():
            if should_force_close_position(pending_trade, business_date):
                blotter.at[i, 'Status'] = 'CANCELED'
                forced_trade = create_forced_trade(business_date,
                                                   pending_trade['Symbol'],
                                                   pending_trade['Action'],
                                                   pending_trade['Size'],
                                                   today_market_data)
                blotter = pd.concat([blotter, pd.DataFrame([forced_trade])], ignore_index=True)
            elif should_close_position(pending_trade, business_date):
                blotter.at[i, 'Status'] = 'FILLED'

# Result & Stats

In [67]:
blotter

,Date,Symbol,Trip,Action,Price,Size,Status
0,2017-01-09 00:00:00,AMZN,ENTRY,SELL,798.0,31.3283,FILLED
1,2017-01-09 00:00:00,WMT,ENTRY,BUY,68.33,365.8715,FILLED
2,2017-01-16 00:00:00,AMZN,EXIT,BUY,711.77,31.3283,CANCELED
3,2017-01-16 00:00:00,WMT,EXIT,SELL,75.71,365.8715,CANCELED
4,2017-01-17 00:00:00,AMZN,EXIT,BUY,809.72,31.3283,FORCED
...,...,...,...,...,...,...,...
780,2022-03-31 00:00:00,WMT,ENTRY,SELL,148.79,168.022,FILLED
781,2022-04-07 00:00:00,AMZN,EXIT,SELL,3716.05,7.51,CANCELED
782,2022-04-07 00:00:00,WMT,EXIT,BUY,131.49,168.022,CANCELED
783,2022-04-08 00:00:00,AMZN,EXIT,SELL,3089.21,7.51,FORCED


In [68]:
def calculate_gain_loss(symbol):
    results = blotter[(blotter['Status'] != 'CANCELED') & (blotter['Symbol'] == symbol)].copy()
    results['Total_Price'] = results['Price'] * results['Size']
    sells = results[results['Action'] == 'SELL'].copy()
    buys = results[results['Action'] == 'BUY'].copy()

    total_sales = sells['Total_Price'].sum()
    total_purchases = buys['Total_Price'].sum()
    gain_loss = total_sales - total_purchases

    print(f"\n******  {symbol}  ******")
    print(f'Total Sales: ${round(total_sales, 2):,}')
    print(f'Total Purchases: ${round(total_purchases, 2):,}')
    print(f'Gain or Loss: ${round(gain_loss, 2):,}')
    return gain_loss


print(f"Entry Orders: {blotter[(blotter['Trip'] == 'ENTRY') & (blotter['Status'] == 'FILLED')].shape[0]}")
print(f"Filled Exit Orders: {blotter[(blotter['Trip'] == 'EXIT') & (blotter['Status'] == 'FILLED')].shape[0]}")
print(f"Forced Exit Orders: {blotter[(blotter['Trip'] == 'EXIT') & (blotter['Status'] == 'FORCED')].shape[0]}")
amz_gain_loss = calculate_gain_loss('AMZN')
wmt_gain_loss = calculate_gain_loss('WMT')
total_gain_loss = amz_gain_loss + wmt_gain_loss
time_period = hist.last_valid_index() - hist.first_valid_index()
years = round(time_period.days / 365.2425, 2)
total_gain_loss_per_year = total_gain_loss / years
print(f"\nYears: {years}")
print(f'Total Gain or Loss: ${round(total_gain_loss, 2):,}')
print(f"Total Gain or Loss Per Year: ${round(total_gain_loss_per_year, 2):,}")

Entry Orders: 264
Filled Exit Orders: 7
Forced Exit Orders: 257

******  AMZN  ******
Total Sales: $3,319,218.3
Total Purchases: $3,301,902.75
Gain or Loss: $17,315.55

******  WMT  ******
Total Sales: $3,308,829.18
Total Purchases: $3,303,183.97
Gain or Loss: $5,645.21

Years: 5.29
Total Gain or Loss: $22,960.76
Total Gain or Loss Per Year: $4,340.41
